#### 数据集
Geolife GPS trajectory dataset
#### idea
聚类得出每类的代表性轨迹，而后计算每条轨迹与其的距离差与时间差（二者看看用什么函数），将距离矩阵投入 Prophet中预测
##### TODO_List
- [x] 聚类（时间差与距离）
- [x] 轨迹平滑处理
   - [x] 代码测试通过
   - [x] 寻找最佳参数--------除了I外的个元素越大越平滑
- [x] 代表性的轨迹
   - [x] 聚类中心--- clusters.centroids[clustersIndex][:,0],clusters.centroids[clustersIndex][:,1]
   - [ ] 轨迹点数目的归一化----Ramer-Douglas-Peucker Algorithm
    - [ ] 归一化参数选择，确保都是一样数目的点
   - [ ] 距离矩阵的计算
   - [ ] 如何仅取ndarray的前2轴
- [ ] Prophet 使用


##### 算法框架
1.  将原始轨迹按 ID 读入内存中，卡尔曼滤波后再进行计算
2.  进行QuickBundles聚类，得出聚类中心线
3.  每类中的所有轨迹与其中心线的距离矩阵的计算
4.  将距离矩阵 + 具体时间点投入Prophet中，注意每个 ID 可能会对应多个预测模型，但每个类仅对应一个预测模型
5.  调参

##### 可能问题
1.  GPS定位不精确导致的位置漂移严重与滞后
2.  新加入轨迹需重新训练
3.  对 ID - 模型的匹配问题
4.  仅能预测有规律的人员出行(上班族)，对随机人群(出租车，销售人员)等无法预测
5.  点间时间跨度可能很大，可能造成轨迹的偏差
6.  离群曲线可能造成聚类较大的偏差

##### 可能的解决方向
1.  点聚类设置 ROI， 并就距离(点与聚类中心)来对点进行拟合(以聚类中心为代表)以减少点的数量和对轨迹进行优化
2.  用较小的 threshold 聚类后， 删除 indices 较少的类别，再进行较高 threshold 的聚类 ( 但可能会损失精度 )


## 数据处理（读取+kalman）

In [ ]:
%matplotlib inline
# 数据处理
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm
from gmplot import gmplot
from pykalman import KalmanFilter
# Enable inline plotting

names = ['lat','lng','zero','alt','days','date','time']
streams_work = []
streams_weekends = []
TimeStamp = []
index = 0
iter = 5
userdata = 'data\\Geolife Trajectories 1.3\\Data\\' + '001' + '\\Trajectory\\'
filelist = os.listdir(userdata)

def func_kalman_2(array):
    measurements = array

    initial_state_mean = [measurements[0, 0],
                      0,
                      measurements[0, 1],
                      0]

    transition_matrix = [[1, 8, 0, 0],
                         [0, 1, 0, 0],
                         [0, 0, 1, 8],
                         [0, 0, 0, 1]]

    observation_matrix = [[1 ,0, 0, 0],
                          [0, 0, 1, 0]]

    kf1 = KalmanFilter(transition_matrices = transition_matrix,
                  observation_matrices = observation_matrix,
                  initial_state_mean = initial_state_mean)
    kf1 = kf1.em(measurements, n_iter=iter)
    (smoothed_state_means, smoothed_state_covariances) = kf1.smooth(measurements)
    kf2 = KalmanFilter(transition_matrices = transition_matrix,
                  observation_matrices = observation_matrix,
                  initial_state_mean = initial_state_mean,
                  observation_covariance = 20*kf1.observation_covariance,
                  em_vars=['transition_covariance', 'initial_state_covariance'])

    kf2 = kf2.em(measurements, n_iter=iter)
    (smoothed_state_means, smoothed_state_covariances)  = kf2.smooth(measurements)
    return smoothed_state_means

for f in tqdm(filelist):
    df_list = [pd.read_csv(userdata + f,header=6,names=names,index_col=False, parse_dates=[['date', 'time']])]
    df = pd.concat(df_list, ignore_index=True)
    df.drop(['zero','alt','days'], axis=1, inplace=True)
    df.set_index('date_time')
    df_min = df.iloc[::12, :]    # 1min

    time_h = df_min['date_time'].apply(lambda x:time.mktime(x.timetuple()))

    smooth = func_kalman_2(np.c_[df_min['lat'].values, df_min['lng'].values])
    lat_lng_data = np.c_[smooth[:,0], smooth[:,2], time_h]
    if df_min['date_time'].dt.dayofweek.mode().values[0] < 5:
        streams_work.append(lat_lng_data)
    else:
        streams_weekends.append(lat_lng_data)

print(streams_work[0][0,0], streams_work[0][0,1], streams_work[0][0,2])

# gmap = gmplot.GoogleMapPlotter(streams[0][0,0], streams[0][0,1], 12)

# for i in range(len(streams)):
#     gmap.plot(streams[i][:,0], streams[i][:,1], 'cornflowerblue', edge_width=1)

# gmap.draw("user001_map.html")
# print('done')




## 分类 (QuickBundles)

In [ ]:
# 分类 1
import geopy.distance
from dipy.segment.metric import Metric
from dipy.segment.metric import ResampleFeature
import numpy as np
from dipy.segment.clustering import QuickBundles

STREAMS = streams_weekends

THRESHOLD = 1

class GPSDistance(Metric):
    def __init__(self):
        super(GPSDistance, self).__init__(feature=ResampleFeature(nb_points=300))

    def are_compatible(self, shape1, shape2):
        return len(shape1) == len(shape2)

    def dist(self, v1, v2):
        x = [geopy.distance.distance([p[0][0], p[0][1]], [p[1][0], p[1][1]]).kilometers  for p in list(zip(v1, v2)) if p[0][2] - p[1][2] < 10]
        currD = np.mean(x)
        return currD

metric = GPSDistance()
qb = QuickBundles(threshold=THRESHOLD, metric=metric)
print (time.strftime('%H:%M:%S',time.localtime(time.time())))
clusters = qb.cluster(STREAMS)
print (time.strftime('%H:%M:%S',time.localtime(time.time())))
print("Nb. clusters:", len(clusters))

In [ ]:
# 分类 - 2
# 调整 threshold 去除仅含1条轨迹的类别
# 双 聚类 代码
import geopy.distance
from dipy.segment.clustering import QuickBundles

from dipy.segment.metric import Metric
from dipy.segment.metric import ResampleFeature
from tqdm import tqdm

STREAMS = streams_work

Min_Trajectory_Number = 1

# 第一次聚类（低threshold 聚多类, 除去离群轨迹)
THRESHOLD_1 = 0.7

class GPSDistance(Metric):
    def __init__(self):
        super(GPSDistance, self).__init__(feature=ResampleFeature(nb_points=300))

    def are_compatible(self, shape1, shape2):
        return len(shape1) == len(shape2)

    def dist(self, v1, v2):
        x = [geopy.distance.distance([p[0][0], p[0][1]], [p[1][0], p[1][1]]).kilometers  for p in list(zip(v1, v2)) if p[0][2] - p[1][2] < 10]
        currD = np.mean(x)
        return currD

metric = GPSDistance()
qb = QuickBundles(threshold=THRESHOLD_1, metric=metric)

clusters = qb.cluster(STREAMS)
print("First Nb. clusters:", len(clusters))

# 去除离群轨迹
Keep_streams = []
mask = clusters > Min_Trajectory_Number
for index in tqdm(range(len(clusters))):
    if mask[index]:
        for i in clusters[index].indices:
            Keep_streams.append(STREAMS[i])


THRESHOLD_2 = 1.3
qb = QuickBundles(threshold=THRESHOLD_2, metric=metric)

clusters_2 = qb.cluster(Keep_streams)
print("Second Nb. clusters:", len(clusters_2))

## 画图（gmplot）

In [ ]:
# 画图 -- 1
from gmplot import gmplot
import random

CLUSTER = clusters

def randomcolor():
    colorArr = ['1','2','3','4','5','6','7','8','9','A','B','C','D','E','F']
    color = ""
    for i in range(6):
        color += colorArr[random.randint(0,14)]
    return "#"+color

gmap = gmplot.GoogleMapPlotter(streams_work[0][0,0], streams_work[0][0,1], 12)

for clustersIndex in range(len(CLUSTER)):
    color = randomcolor()
    for i in CLUSTER[clustersIndex].indices:
        gmap.plot(streams_work[i][:,0], streams_work[i][:,1], color, edge_width=1)

gmap.draw("user001_map.html")
print('Map done')


In [ ]:
## 画图 -- 2

from matplotlib import rcParams

rcParams['figure.facecolor']=(1,1,1,1)
# i = 0
clusters_sample = clusters_2
for i in range(len(clusters_sample)):
#     plt.plot(clusters_2.centroids[i][:,0], clusters_2.centroids[i][:,1], label='centroid')
    for j in clusters_sample[i].indices:
        plt.plot(streams_work[j][:,0], streams_work[j][:,1])

# for i in range(len(clusters_2)):
#     plt.plot(clusters_2.centroids[i][:,0], clusters_2.centroids[i][:,1], label='centroid')
#     for j in clusters_2[i].indices:
#         plt.plot(streams_work[j][:,0], streams_work[j][:,1]) # Plot some data on the (implicit) axes.

# plt.xlabel('latitude')
# plt.ylabel('longitude')
# plt.title("Simple Plot")
# plt.legend()

In [ ]:
from datetime import datetime

def center_change_time_type(centroids):

    Test_Array = centroids

    dt1 = pd.date_range(start='2020-01-01',end='2020-01-02',freq="4.8T") # 300个点，与上文的Resample一致, 时间随意，因为不会用到
    pydate_array = dt1.to_pydatetime()
    pydate_array = np.delete(pydate_array,-1)
    Test_Array = np.delete(Test_Array, -1, axis=1)
    Test_Array = np.c_[Test_Array, pydate_array]

    return Test_Array

def stream_change_time_type(stream):

    temp_var = np.vectorize(lambda s: datetime.fromtimestamp(time.mktime(time.localtime(s))))(stream[:,2])
    temp_array = np.delete(stream, -1, axis=1)
    stream = np.c_[temp_array, temp_var]

    return stream

def find_nearest_time_point(centroids, stream):

    index_of_point = np.argmin(np.vectorize(lambda t, s: (t-s).seconds)(center_change_time_type(centroids)[:,2], stream_change_time_type(stream)[0,2]))

    if centroids.shape[0] - index_of_point >= stream.shape[0]:
        is_longer_than_stream = True
    else:
        is_longer_than_stream = False

    return index_of_point, is_longer_than_stream

print(clusters_2.centroids[0].shape)
print(streams_work[0].shape)
print(np.vectorize(lambda x:time.strftime('%Y/%m/%d %H:%M:%S',time.localtime(x)))(streams_work[0][0,2]))
print(clusters_2.centroids[0])
index_of_point , is_longer_than_stream = find_nearest_time_point(clusters_2.centroids[0], streams_work[0])

In [ ]:
from datetime import datetime
from tqdm import tqdm

distance_list = []
time_list = []
def center_change_time_type(centroids):

    Test_Array = centroids

    dt1 = pd.date_range(start='2020-01-01',end='2020-01-02',freq="4.8T") # 300个点，与上文的Resample一致, 时间随意，因为不会用到
    pydate_array = dt1.to_pydatetime()
    pydate_array = np.delete(pydate_array,-1)
    Test_Array = np.delete(Test_Array, -1, axis=1)
    Test_Array = np.c_[Test_Array, pydate_array]

    return Test_Array

def stream_change_time_type(stream):

    temp_var = np.vectorize(lambda s: datetime.fromtimestamp(time.mktime(time.localtime(s))))(stream[:,2])
    temp_array = np.delete(stream, -1, axis=1)
    stream = np.c_[temp_array, temp_var]

    return stream

def find_nearest_time_point(centroids, stream):

    index_of_point = np.argmin(np.vectorize(lambda t, s: (t-s).seconds)(center_change_time_type(centroids)[:,2], stream_change_time_type(stream)[0,2]))

    if centroids.shape[0] - index_of_point >= stream.shape[0]:
        is_longer_than_stream = True
    else:
        is_longer_than_stream = False

    return index_of_point, is_longer_than_stream

for i in tqdm(clusters_2[3].indices):

    index_of_point , is_longer_than_stream = find_nearest_time_point(clusters_2.centroids[0], streams_work[i])

    v1 = clusters_2.centroids[0][index_of_point:,:]

    if is_longer_than_stream:
        v2 = streams_work[i]
    else:
        v2 = streams_work[i][:v1.shape[0],:]

    x = [geopy.distance.distance([p[0][0], p[0][1]], [p[1][0], p[1][1]]).kilometers for p in list(zip(v1, v2))]
    # x = np.c_[x,v2[:,2]]
    distance_list = distance_list + x
    time_list = time_list + np.vectorize(lambda x:time.strftime('%H:%M:%S',time.localtime(x)))(v2[:,2]).tolist()

print(len(distance_list), len(time_list))

In [ ]:
from matplotlib import rcParams
print(time_list[0] , time_list[300] , time_list[430])
rcParams['figure.facecolor']=(1,1,1,1)
x = time_list
y = distance_list
plt.scatter(x, y, alpha=0.6)  # 绘制散点图，透明度为0.6（这样颜色浅一点，比较好看）
plt.show()

In [ ]:

mm=pd.DataFrame(time_list,columns=['ds'])
mm=pd.concat([mm,pd.DataFrame(distance_list,columns=['y'])],axis=1)

x = mm['y']
y = mm['ds']
plt.scatter(x, y, alpha=0.6)  # 绘制散点图，透明度为0.6（这样颜色浅一点，比较好看）
plt.show()

In [ ]:
import pandas as pd
from fbprophet import Prophet
# df = pd.read_csv('data\example_wp_log_peyton_manning.csv')
# df.head()
df = mm
m = Prophet()
m.fit(df)

future = m.make_future_dataframe(periods=1, freq='H')
fcst = m.predict(future)
print(fcst[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

fig = m.plot(fcst)
fig = m.plot_components(fcst)

In [ ]:
# import numpy as np
# import pandas as pd
#
#
# def find_nearest(centroids, stream):
#     array = np.asarray(centroids)
#     Longer_array = -1 # 1 -> left longer , 0 -> right longer
#     if array.shape[0] >= stream.shape[0]:
#         idx = (np.abs(array[:,2] - stream[0,2])).argmin()
#         Longer_array = 1
#     else:
#         idx = (np.abs(array[0,2] - stream[:,2])).argmin()
#         Longer_array = 1
#     return idx, Longer_array
#
# for i in clusters_2[0].indices:
#
#     index, Longer_Array_Test_Array = find_nearest(Test_Array, streams_work[i]) # 轨迹对齐
#
#     if Longer_Array_Test_Array:
#         for center_point , stream_point in zip(Test_Array[index:,:],streams_work[i]):
#             # geopy.distance.distance([p[0][0], p[0][1]], [p[1][0], p[1][1]]).kilometers  for p in list(zip(clusters_2.centroids[0], stream))
#             print("HERE!!!")
#             print(center_point , stream_point)
